In [ ]:
import pandas as pd
import os
import duckdb
from tqdm import tqdm

In [ ]:
# Caminho da pasta onde estão os arquivos
pasta = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\efd_base\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\Nova pasta'


In [25]:
tabela_completa = []


for arquivo in os.listdir(pasta):
    if arquivo.endswith('.xlsx') and 'Entrada' in arquivo:
        caminho_arquivo = os.path.join(pasta, arquivo)
        print (f'Lendo {arquivo}')
        df = pd.read_excel(caminho_arquivo, sheet_name='Dados EFD Entrada',skiprows=1)
        tabela_completa.append(df)


df_final = pd.concat(tabela_completa, ignore_index=True)
tipos = {
    'Inscrição Estadual': 'string',
    'CPF/CNPJ': 'string',
    'Ano de referência': 'string',
    'Mês de apuração': 'string',
    'Hash do arquivo transmitido': 'string',
    'Origem da Informação': 'string',
    'Cód. Município Origem D100': 'string',
    'CFOP': 'string',
    # 'Valor Operação': 'float',
    # 'Valor BC ICMS': 'float',
    'Declaração utilizado?': 'string',
    'Registro utilizado no calculo VA?': 'string'
}
df_final = df_final.astype(tipos)


df_final.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\efd_base\\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\\efd_base_entradas.parquet')


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte3_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
df = pd.read_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\efd_base\\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\\efd_base_entradas.parquet')


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333109 entries, 0 to 333108
Data columns (total 12 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Inscrição Estadual                 333109 non-null  string
 1   CPF/CNPJ                           333109 non-null  string
 2   Ano de referência                  333109 non-null  string
 3   Mês de apuração                    333109 non-null  string
 4   Hash do arquivo transmitido        333109 non-null  string
 5   Origem da Informação               333109 non-null  string
 6   Cód. Município Origem D100         333034 non-null  string
 7   CFOP                               333109 non-null  string
 8   Valor Operação                     333109 non-null  object
 9   Valor BC ICMS                      333046 non-null  object
 10  Declaração utilizado?              333109 non-null  string
 11  Registro utilizado no calculo VA?  333109 non-null  

In [28]:
df

,Inscrição Estadual,CPF/CNPJ,Ano de referência,Mês de apuração,Hash do arquivo transmitido,Origem da Informação,Cód. Município Origem D100,CFOP,Valor Operação,Valor BC ICMS,Declaração utilizado?,Registro utilizado no calculo VA?
0,14587535,11137051092102,2025,2,0ABB8048AADA0107B33F39A5E5D11DEF,C190,0.0,2102,"2.303,98","0,00",SIM,SIM
1,14587535,11137051092102,2025,2,0ABB8048AADA0107B33F39A5E5D11DEF,C190,0.0,2407,"225,12","0,00",SIM,NÃO
2,14587535,11137051092102,2025,2,0ABB8048AADA0107B33F39A5E5D11DEF,C190,0.0,2556,"7.057,88","0,00",SIM,NÃO
3,14587535,11137051092102,2025,2,0ABB8048AADA0107B33F39A5E5D11DEF,C590,0.0,1253,"1.338,06","0,00",SIM,NÃO
4,14587535,11137051092102,2025,2,0ABB8048AADA0107B33F39A5E5D11DEF,D190,3304557.0,1353,"243,51","0,00",SIM,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...
333104,78198290,8409412000191,2025,6,CEDFE429A26D87EB37717AECC6679629,D190,4126256.0,2352,"252,40","248,77",SIM,SIM
333105,78198290,8409412000191,2025,6,CEDFE429A26D87EB37717AECC6679629,D190,4208203.0,2352,"10.925,54","10.925,54",SIM,SIM
333106,78198290,8409412000191,2025,6,CEDFE429A26D87EB37717AECC6679629,D190,4206306.0,2352,"1.881,88","0,00",SIM,SIM
333107,78198290,8409412000191,2025,6,CEDFE429A26D87EB37717AECC6679629,D190,2900702.0,2352,"194,50","194,50",SIM,SIM


In [1]:
import os

def create_gitkeep_files(root_path, folders):
    for folder in folders:
        folder_path = os.path.join(root_path, folder)
        os.makedirs(folder_path, exist_ok=True)  # cria pasta se não existir
        gitkeep_path = os.path.join(folder_path, '.gitkeep')
        with open(gitkeep_path, 'w') as f:
            f.write('')  # cria arquivo .gitkeep vazio

# Caminho raiz do seu repositório local
root_path = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025'

# Lista de pastas relativas dentro do repositório
folders = [
    'data/dados_cadastrais',
    'data/efd_base',
    'data/efd_entrega',
    'data/pgdas',
    'data/valor_adicionado'
]

create_gitkeep_files(root_path, folders)
